## Explore the json file

In [1]:
## Reading and importing the no_tax json file
import json
with open("no_tax.json", 'r') as file:
    tax_json = json.load(file)

In [ ]:
## Finding the fields within the json file
fields = tax_json['meta']['view']['columns']
for field in fields:
    print(field['name'])


sid
id
position
created_at
created_meta
updated_at
updated_meta
meta
the_geom
OBJECTID
ID
Cluster Letter
Shape.STArea()
Shape.STLength()


In [3]:
## Covnverting the WKT to polygons
import arcpy
arcpy.FromWKT(tax_json['data'][8][8]) 
for row in tax_json['data']:
    row[8] = arcpy.FromWKT (row[8]) 
    print(row)

['row-69eh-dt2h-vwz3', '00000000-0000-0000-A344-B176ECD7FE9B', 0, 1628101573, None, 1628101573, None, '{ }', <Polygon object at 0x1f19009cc40[0x1f19009cab0]>, '101', '220710050002', 'C', '3085170.5967876972', '7251.2480743083825']
['row-7new-5v4m~u4mk', '00000000-0000-0000-B0F6-DB2ECA268590', 0, 1628101573, None, 1628101573, None, '{ }', <Polygon object at 0x1f18ff96550[0x1f19009c480]>, '317', '220710007012', 'G', '2315413.2514727716', '8195.0211310674586']
['row-wgta_kfdc~mtyi', '00000000-0000-0000-9D07-2EB6550E4D75', 0, 1628101573, None, 1628101573, None, '{ }', <Polygon object at 0x1f199e28700[0x1f199e286c0]>, '312', '220710024022', 'F', '3253985.2138586314', '8043.8126011148997']
['row-qxtd-kf7g_8itj', '00000000-0000-0000-8D34-BF0D2E55747B', 0, 1628101573, None, 1628101573, None, '{ }', <Polygon object at 0x1f199e28790[0x1f199e28750]>, '8', '220710013022', 'F', '1597228.8210011073', '5644.5249819895553']
['row-swzb.ccqd_5u7d', '00000000-0000-0000-AA94-02D6A5EFB806', 0, 1628101573, 

## Create the feature class and assin fields

In [28]:
#Setting up the workspace
import os

fcname = 'notax_fc.shp'
workspace = r'c:\Users\carol\Documents\Geog 4057 Final'
fc_fullname = os.path.join(workspace, fcname)
## Checking if file already exists so a duplicate is not created
if arcpy.Exists(fc_fullname):
    arcpy.management.Delete(fc_fullname)

arcpy.management.CreateFeatureclass(out_path=workspace, out_name=fcname, geometry_type='POLYGON', spatial_reference=4236)

desc = arcpy.da.Describe(fc_fullname)
for field in desc['fields']:
    print(field.name)

FID
Shape
Id


In [29]:
# Finding each field and defining its type
fields = tax_json['meta']['view']['columns']
for field in fields:
    print(field['name'])
field_type = ['TEXT', 'TEXT', 'LONG', 'LONG', 'TEXT', 'LONG', 'TEXT', 'TEXT', 'TEXT','TEXT','TEXT','TEXT','TEXT',]
field_names=[]
for ind,field in enumerate(fields):
    name = field['name']
    # Exluding the polygon field
    if name == 'the_geom':
        continue
    # Preventing repeat names
    if name.lower() == 'id':
        name = f'id_{ind}'
    max_len = min(10,len(name))
    name = name[:max_len]
    field_names.append(name)
# Removing spaces and periods from names 
field_names = [field.replace(" ","_") for field in field_names]
field_names = [field.replace(".","_") for field in field_names]
field_names



sid
id
position
created_at
created_meta
updated_at
updated_meta
meta
the_geom
OBJECTID
ID
Cluster Letter
Shape.STArea()
Shape.STLength()


['sid',
 'id_1',
 'position',
 'created_at',
 'created_me',
 'updated_at',
 'updated_me',
 'meta',
 'OBJECTID',
 'id_10',
 'Cluster_Le',
 'Shape_STAr',
 'Shape_STLe']

In [30]:
#Checking field names
print(field_names)

['sid', 'id_1', 'position', 'created_at', 'created_me', 'updated_at', 'updated_me', 'meta', 'OBJECTID', 'id_10', 'Cluster_Le', 'Shape_STAr', 'Shape_STLe']


In [32]:
#Creating the fields
for ind,field_name in enumerate(field_names):
    arcpy.management.AddField(fc_fullname,field_name=field_name,field_type=field_type[ind])


In [36]:
field_names.append('SHAPE@')

In [37]:
len(field_names)

14

## Add data to the feature class

In [38]:
with arcpy.da.InsertCursor(fc_fullname,field_names=field_names) as cursor:
    for row in tax_json['data']:
        new_row = []
        for ind, value in enumerate(row):
            if ind == 8:
                continue
            if value == None:
                value = ''
            new_row.append(value)
        new_row.append(row[8])
        cursor.insertRow(new_row)


In [34]:
  for row in tax_json['data']:
        print(row)

['row-69eh-dt2h-vwz3', '00000000-0000-0000-A344-B176ECD7FE9B', 0, 1628101573, None, 1628101573, None, '{ }', <Polygon object at 0x1f19009cc40[0x1f19009cab0]>, '101', '220710050002', 'C', '3085170.5967876972', '7251.2480743083825']
['row-7new-5v4m~u4mk', '00000000-0000-0000-B0F6-DB2ECA268590', 0, 1628101573, None, 1628101573, None, '{ }', <Polygon object at 0x1f18ff96550[0x1f19009c480]>, '317', '220710007012', 'G', '2315413.2514727716', '8195.0211310674586']
['row-wgta_kfdc~mtyi', '00000000-0000-0000-9D07-2EB6550E4D75', 0, 1628101573, None, 1628101573, None, '{ }', <Polygon object at 0x1f199e28700[0x1f199e286c0]>, '312', '220710024022', 'F', '3253985.2138586314', '8043.8126011148997']
['row-qxtd-kf7g_8itj', '00000000-0000-0000-8D34-BF0D2E55747B', 0, 1628101573, None, 1628101573, None, '{ }', <Polygon object at 0x1f199e28790[0x1f199e28750]>, '8', '220710013022', 'F', '1597228.8210011073', '5644.5249819895553']
['row-swzb.ccqd_5u7d', '00000000-0000-0000-AA94-02D6A5EFB806', 0, 1628101573, 